In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# Training loop
```
Initialise model parameters ---> Input data --> create model --> compute loss ---> 
         |                                                                        | 
         -----------------<---------- Adjust parameters ---------<----------------- 
```

1. Input data
2. Create model
3. Create loss function
4. Create minimiser
5. Create a TensorFlow session
6. Loop the train operation until convergence criterion
7. Save model

In [2]:
from sklearn.datasets import load_digits
from sklearn.metrics import roc_auc_score, accuracy_score
mnist = load_digits(n_class = 2)

# Logistic Regression 

In [3]:
#initalise variables / model parameters
X, y = mnist.data,mnist.target
print(f'X dim : {X.shape}')
print(f'y length : {len(y)}')
print(f'Target example : {y[0:10]}')

num_of_features = X.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 56)
print(f'X train dim : {X_train.shape}')
print(f'y train length : {len(y_train)}')
print(f'Train target example : {y_train[0:10]}')

X dim : (360, 64)
y length : 360
Target example : [0 1 0 1 0 1 0 0 1 1]
X train dim : (270, 64)
y train length : 270
Train target example : [1 0 0 1 0 0 1 1 0 0]


In [4]:
type(y_train[0])

numpy.int64

In [5]:
W = tf.Variable (tf.zeros(shape = [num_of_features, 1]) , dtype=tf.float32,name = 'weights' )
b = tf.Variable (0,dtype=tf.float32, name = 'bias')

In [6]:
learning_rate = 1e-3

In [7]:
#define the training loop operations
def inference(X):
    """
    compute inference model over data (X) 
    and return the result
    """
    tmp = tf.matmul(X, W) + b
    predicted_y = tf.squeeze(tf.nn.sigmoid(tmp))
    
    return predicted_y

In [8]:
#define a loss function
def loss(X,Y, fun = 'mean_square'):
    """
    compute loss over data (X) and expected output (Y)
    """
    Y_predicted = inference(X)
    if fun == 'mean_square':
        loss =  tf.reduce_sum(tf.squared_difference(Y,Y_predicted))
    elif fun == 'cross_entropy':
        loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(Y,Y_predicted))
        
    return loss        

In [9]:
def inputs():
    """
    read/generate training data X and labels Y
    """
    input_X = tf.placeholder(tf.float32,[None, num_of_features],name = 'input_data')
    input_Y = tf.placeholder(dtype=tf.float32,shape=[None],name = 'data_labels')
    return input_X,input_Y 

In [10]:
def train(total_loss):
    """train/adjust model parameters according to total loss"""
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    return optimizer

In [11]:
def evaluate(sess, X,Y):
    """evaluate the model""" 
    y_pred = tf.cast(inference(X),tf.float32)
    res = sess.run(tf.reduce_mean(tf.cast(tf.equal(Y,y_pred),tf.float32)))
    return res

In [12]:
#tf.reset_default_graph()
saver = tf.train.Saver()
with tf.Session() as sess:
    model_variables = tf.global_variables_initializer() 
    sess.run(model_variables)
    #tf.global_variables_initializer().run()
    
    X,Y = inputs()
    total_loss = loss(X,Y)
    train_op = train(total_loss)
    
    ###########################
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord = coord)
    ###########################
    num_of_steps = 100
    
    pred_y = inference(X)
    
    for step in range(num_of_steps):
        sess.run(train_op,{X: X_train, Y: y_train})
        if step%10==0:
            print ("loss {}".format(sess.run(total_loss,{X: X_train, Y: y_train})))
            print("train auc:", roc_auc_score(y_train, sess.run(pred_y, {X:X_train})))
            print("train accuracy:", accuracy_score(y_train, sess.run(pred_y, {X:X_train}).round()))
            print("test auc:", roc_auc_score(y_test, sess.run(pred_y, {X:X_test})))
            print("test accuracy:", accuracy_score(y_test, sess.run(pred_y, {X:X_test}).round()),end='\n====\n')
            #loss reduces as it compares probabilities, while accuracy remains the same.
            saver.save(sess,'model',global_step = step)
            
            
    coord.request_stop()

loss 0.7504878640174866
train auc: 1.0
train accuracy: 0.996296296296
test auc: 0.99950617284
test accuracy: 0.988888888889
====
loss 0.02127673104405403
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.01470182090997696
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.011650933884084225
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.009653536602854729
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.00823899731040001
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.007185106631368399
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.0063697295263409615
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.988888888889
====
loss 0.005720335058867931
train auc: 1.0
train accuracy: 1.0
test auc: 1.0
test accuracy: 0.98888888888